In [1]:
# Import packages
# dxpy allows python to interact with the platform storage
# Note: This notebook is using spark since the size of the dataset we're extracting
# (i.e. the number of fields) is too large for a single node instance.
import dxdata
import dxpy
import pandas as pd
import subprocess
import glob
import os
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [2]:
os.getcwd()

'/opt/notebooks'

In [3]:
# Automatically discover dispensed dataset ID
dispensed_dataset = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)
dispensed_dataset_id = dispensed_dataset["id"]

In [4]:
# Get project ID
project_id = dxpy.find_one_project()["id"]

In [5]:
dataset = (":").join([project_id, dispensed_dataset_id])

In [6]:
# Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run
cmd = ["dx", "extract_dataset", dataset, "-ddd", "--delimiter", ","]
subprocess.check_call(cmd)

0

In [7]:
path = os.getcwd()

In [8]:
data_dict_csv = glob.glob(os.path.join(path, "*.data_dictionary.csv"))[0]
data_dict_df = pd.read_csv(data_dict_csv)
data_dict_df.head()

/tmp/ipykernel_119/118131557.py:2: DtypeWarning: Columns (4,7,8,9,10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_dict_df = pd.read_csv(data_dict_csv)


,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
0,participant,eid,string,global,NaN,NaN,NaN,Participant Information,NaN,NaN,NaN,NaN,NaN,NaN,Participant ID,NaN
1,participant,p3_i0,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 0,seconds
2,participant,p3_i1,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 1,seconds
3,participant,p3_i2,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 2,seconds
4,participant,p3_i3,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 3,seconds


In [9]:
field_names = list(
    data_dict_df.loc[data_dict_df["entity"] == "olink_instance_1", "name"].values
)
print(len(field_names))

0


In [10]:
field_names = list(
    data_dict_df.loc[data_dict_df["entity"] == "olink_instance_0", "name"].values
)
print(len(field_names))

2924


In [11]:
#Olink data has only one instance

In [12]:
# Need to adjust this buffer otherwise will get an error in toPandas() call
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "256m")

In [15]:
!pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 196.1 MB/s eta 0:00:00


In [13]:
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
sqlContext = SQLContext(sc)

/cluster/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [16]:
metabo1 = pd.read_html("https://biobank.ndph.ox.ac.uk/showcase/label.cgi?id=220")

In [17]:
metabo_df1 = metabo1[1]
metabo_df1.head()

,Field ID,Description
0,23474,3-Hydroxybutyrate
1,23475,Acetate
2,23476,Acetoacetate
3,23477,Acetone
4,23460,Alanine


In [18]:
field_list = metabo_df1['Field ID'].to_list()

In [19]:
metabo_fields = ['p' + str(element) + '_i1' for element in field_list]

In [20]:
# Load the original dataset - record-id can be obtained from inspecting details of the database in the UI
dataset = dxdata.load_dataset(id='record-Gq77990JqZXKQB0PzqKXfXxy')

In [21]:
# Load your pre-filtered cohort
cont = dxdata.load_cohort(folder="/", name="all_participants")

In [22]:
participant = dataset["participant"]

In [24]:
metabo_fields2 = ['eid', metabo_fields]
metabo_fields3 = [metabo_fields2[0]] + metabo_fields2[1]
#metabo_fields3

In [25]:
cont_df = participant.retrieve_fields(names=metabo_fields3, filter_sql=cont.sql, engine=dxdata.connect()).toPandas()
cont_df.head()

/opt/conda/lib/python3.11/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl_version)


,eid,p23474_i1,p23475_i1,p23476_i1,p23477_i1,p23460_i1,p23479_i1,p23440_i1,p23439_i1,p23441_i1,...,p23598_i1,p23648_i1,p23628_i1,p23603_i1,p23633_i1,p23588_i1,p23608_i1,p23469_i1,p23403_i1,p23467_i1
0,1000028,0.075385,0.025734,0.034807,0.011765,0.33815,38.037,1.6385,0.63813,0.38947,...,73.951,7.6781,14.502,58.743,15.777,64.857,36.351,0.087856,0.7171,0.26575
1,1000045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
cont_df2 = cont_df.dropna(subset=cont_df.columns[1:],how='all')
cont_df2.head()

,eid,p23474_i1,p23475_i1,p23476_i1,p23477_i1,p23460_i1,p23479_i1,p23440_i1,p23439_i1,p23441_i1,...,p23598_i1,p23648_i1,p23628_i1,p23603_i1,p23633_i1,p23588_i1,p23608_i1,p23469_i1,p23403_i1,p23467_i1
0,1000028,0.075385,0.025734,0.034807,0.011765,0.33815,38.037,1.6385,0.63813,0.38947,...,73.951,7.6781,14.5020,58.743,15.7770,64.857,36.351,0.087856,0.71710,0.26575
23,1001019,0.051107,0.026065,0.013434,0.010894,0.34169,38.676,1.3146,0.71974,0.54750,...,65.246,6.0583,8.0868,51.120,26.9340,63.214,31.341,0.062773,0.64341,0.21705
197,1009261,0.030850,0.038164,0.013678,0.010973,0.25740,41.523,1.3825,1.08950,0.78807,...,47.161,5.3529,5.5195,38.748,5.1343,52.856,19.061,0.056372,1.03870,0.20597
201,1009359,0.244580,0.038729,0.045615,0.058788,0.20749,43.077,1.4790,1.09200,0.73835,...,34.045,3.6376,3.5326,24.501,3.0541,42.657,14.593,0.063465,0.86576,0.23846
220,1010306,0.011809,0.020613,0.015611,0.010041,0.26239,41.903,1.6107,0.77259,0.47965,...,56.700,6.9960,10.3320,44.917,23.8190,56.903,27.297,0.067967,0.81552,0.22096


In [27]:
cont_df2.shape

(16703, 252)

In [29]:
prefix = 'p'
suffix = '_i1'

In [30]:
metabo_df1['Field ID'] = metabo_df1['Field ID'].astype(str).apply(lambda x: f"{prefix}{x}{suffix}")
metabo_df1.head()

,Field ID,Description
0,p23474_i1,3-Hydroxybutyrate
1,p23475_i1,Acetate
2,p23476_i1,Acetoacetate
3,p23477_i1,Acetone
4,p23460_i1,Alanine


In [31]:
column_dict = dict(zip(metabo_df1.iloc[:, 0], metabo_df1.iloc[:, 1]))

In [32]:
cont_df3 = cont_df2.rename(columns=column_dict)
cont_df3.head()

,eid,3-Hydroxybutyrate,Acetate,Acetoacetate,Acetone,Alanine,Albumin,Apolipoprotein A1,Apolipoprotein B,Apolipoprotein B to Apolipoprotein A1 ratio,...,Triglycerides to Total Lipids in Medium VLDL percentage,Triglycerides to Total Lipids in Small HDL percentage,Triglycerides to Total Lipids in Small LDL percentage,Triglycerides to Total Lipids in Small VLDL percentage,Triglycerides to Total Lipids in Very Large HDL percentage,Triglycerides to Total Lipids in Very Large VLDL percentage,Triglycerides to Total Lipids in Very Small VLDL percentage,Tyrosine,VLDL Cholesterol,Valine
0,1000028,0.075385,0.025734,0.034807,0.011765,0.33815,38.037,1.6385,0.63813,0.38947,...,73.951,7.6781,14.5020,58.743,15.7770,64.857,36.351,0.087856,0.71710,0.26575
23,1001019,0.051107,0.026065,0.013434,0.010894,0.34169,38.676,1.3146,0.71974,0.54750,...,65.246,6.0583,8.0868,51.120,26.9340,63.214,31.341,0.062773,0.64341,0.21705
197,1009261,0.030850,0.038164,0.013678,0.010973,0.25740,41.523,1.3825,1.08950,0.78807,...,47.161,5.3529,5.5195,38.748,5.1343,52.856,19.061,0.056372,1.03870,0.20597
201,1009359,0.244580,0.038729,0.045615,0.058788,0.20749,43.077,1.4790,1.09200,0.73835,...,34.045,3.6376,3.5326,24.501,3.0541,42.657,14.593,0.063465,0.86576,0.23846
220,1010306,0.011809,0.020613,0.015611,0.010041,0.26239,41.903,1.6107,0.77259,0.47965,...,56.700,6.9960,10.3320,44.917,23.8190,56.903,27.297,0.067967,0.81552,0.22096


In [33]:
cont_df3['FID'] = cont_df3['eid']
cont_df3['IID'] = cont_df3['eid']
cont_df3.head()

,eid,3-Hydroxybutyrate,Acetate,Acetoacetate,Acetone,Alanine,Albumin,Apolipoprotein A1,Apolipoprotein B,Apolipoprotein B to Apolipoprotein A1 ratio,...,Triglycerides to Total Lipids in Small LDL percentage,Triglycerides to Total Lipids in Small VLDL percentage,Triglycerides to Total Lipids in Very Large HDL percentage,Triglycerides to Total Lipids in Very Large VLDL percentage,Triglycerides to Total Lipids in Very Small VLDL percentage,Tyrosine,VLDL Cholesterol,Valine,FID,IID
0,1000028,0.075385,0.025734,0.034807,0.011765,0.33815,38.037,1.6385,0.63813,0.38947,...,14.5020,58.743,15.7770,64.857,36.351,0.087856,0.71710,0.26575,1000028,1000028
23,1001019,0.051107,0.026065,0.013434,0.010894,0.34169,38.676,1.3146,0.71974,0.54750,...,8.0868,51.120,26.9340,63.214,31.341,0.062773,0.64341,0.21705,1001019,1001019
197,1009261,0.030850,0.038164,0.013678,0.010973,0.25740,41.523,1.3825,1.08950,0.78807,...,5.5195,38.748,5.1343,52.856,19.061,0.056372,1.03870,0.20597,1009261,1009261
201,1009359,0.244580,0.038729,0.045615,0.058788,0.20749,43.077,1.4790,1.09200,0.73835,...,3.5326,24.501,3.0541,42.657,14.593,0.063465,0.86576,0.23846,1009359,1009359
220,1010306,0.011809,0.020613,0.015611,0.010041,0.26239,41.903,1.6107,0.77259,0.47965,...,10.3320,44.917,23.8190,56.903,27.297,0.067967,0.81552,0.22096,1010306,1010306


In [34]:
cont_df4 = cont_df3.drop('eid',axis=1)
cont_df4.head()

,3-Hydroxybutyrate,Acetate,Acetoacetate,Acetone,Alanine,Albumin,Apolipoprotein A1,Apolipoprotein B,Apolipoprotein B to Apolipoprotein A1 ratio,Average Diameter for HDL Particles,...,Triglycerides to Total Lipids in Small LDL percentage,Triglycerides to Total Lipids in Small VLDL percentage,Triglycerides to Total Lipids in Very Large HDL percentage,Triglycerides to Total Lipids in Very Large VLDL percentage,Triglycerides to Total Lipids in Very Small VLDL percentage,Tyrosine,VLDL Cholesterol,Valine,FID,IID
0,0.075385,0.025734,0.034807,0.011765,0.33815,38.037,1.6385,0.63813,0.38947,9.5859,...,14.5020,58.743,15.7770,64.857,36.351,0.087856,0.71710,0.26575,1000028,1000028
23,0.051107,0.026065,0.013434,0.010894,0.34169,38.676,1.3146,0.71974,0.54750,9.3226,...,8.0868,51.120,26.9340,63.214,31.341,0.062773,0.64341,0.21705,1001019,1001019
197,0.030850,0.038164,0.013678,0.010973,0.25740,41.523,1.3825,1.08950,0.78807,9.6360,...,5.5195,38.748,5.1343,52.856,19.061,0.056372,1.03870,0.20597,1009261,1009261
201,0.244580,0.038729,0.045615,0.058788,0.20749,43.077,1.4790,1.09200,0.73835,9.5342,...,3.5326,24.501,3.0541,42.657,14.593,0.063465,0.86576,0.23846,1009359,1009359
220,0.011809,0.020613,0.015611,0.010041,0.26239,41.903,1.6107,0.77259,0.47965,9.5071,...,10.3320,44.917,23.8190,56.903,27.297,0.067967,0.81552,0.22096,1010306,1010306


In [35]:
cont_df5 = cont_df4.iloc[:, [251,252] + list(range(251))]
cont_df5.head()

,FID,IID,3-Hydroxybutyrate,Acetate,Acetoacetate,Acetone,Alanine,Albumin,Apolipoprotein A1,Apolipoprotein B,...,Triglycerides to Total Lipids in Medium VLDL percentage,Triglycerides to Total Lipids in Small HDL percentage,Triglycerides to Total Lipids in Small LDL percentage,Triglycerides to Total Lipids in Small VLDL percentage,Triglycerides to Total Lipids in Very Large HDL percentage,Triglycerides to Total Lipids in Very Large VLDL percentage,Triglycerides to Total Lipids in Very Small VLDL percentage,Tyrosine,VLDL Cholesterol,Valine
0,1000028,1000028,0.075385,0.025734,0.034807,0.011765,0.33815,38.037,1.6385,0.63813,...,73.951,7.6781,14.5020,58.743,15.7770,64.857,36.351,0.087856,0.71710,0.26575
23,1001019,1001019,0.051107,0.026065,0.013434,0.010894,0.34169,38.676,1.3146,0.71974,...,65.246,6.0583,8.0868,51.120,26.9340,63.214,31.341,0.062773,0.64341,0.21705
197,1009261,1009261,0.030850,0.038164,0.013678,0.010973,0.25740,41.523,1.3825,1.08950,...,47.161,5.3529,5.5195,38.748,5.1343,52.856,19.061,0.056372,1.03870,0.20597
201,1009359,1009359,0.244580,0.038729,0.045615,0.058788,0.20749,43.077,1.4790,1.09200,...,34.045,3.6376,3.5326,24.501,3.0541,42.657,14.593,0.063465,0.86576,0.23846
220,1010306,1010306,0.011809,0.020613,0.015611,0.010041,0.26239,41.903,1.6107,0.77259,...,56.700,6.9960,10.3320,44.917,23.8190,56.903,27.297,0.067967,0.81552,0.22096


In [36]:
cont_df5.to_csv('metabolomics_pheno_2nd_instance_for_pravesh.csv',sep='\t',index=False,na_rep='NA')

In [48]:
field2 = ['eid','p52','p34','p23658_i0','p23658_i1']

In [49]:
measure_date1 = participant.retrieve_fields(names=field2, filter_sql=cont.sql, engine=dxdata.connect()).toPandas()
measure_date1.head()

,eid,p52,p34,p23658_i0,p23658_i1
0,1000028,2,1955,2019-09-25 21:18:41,2021-06-23 07:21:04
1,1000045,1,1947,NaT,NaT
2,1000104,6,1953,NaT,NaT
3,1000171,6,1945,2020-03-05 12:19:11,NaT
4,1000196,6,1944,NaT,NaT


In [50]:
measure_date2 = measure_date1.rename(columns={'eid':'FID','p52':'Birth_Month','p34':'Birth_Year','p23658_i0':'Measurement_date_i0','p23658_i1':'Measurement_date_i1'})
measure_date2.head()

,FID,Birth_Month,Birth_Year,Measurement_date_i0,Measurement_date_i1
0,1000028,2,1955,2019-09-25 21:18:41,2021-06-23 07:21:04
1,1000045,1,1947,NaT,NaT
2,1000104,6,1953,NaT,NaT
3,1000171,6,1945,2020-03-05 12:19:11,NaT
4,1000196,6,1944,NaT,NaT


In [51]:
measure_date2['IID'] = measure_date2['FID']
measure_date2.head()

,FID,Birth_Month,Birth_Year,Measurement_date_i0,Measurement_date_i1,IID
0,1000028,2,1955,2019-09-25 21:18:41,2021-06-23 07:21:04,1000028
1,1000045,1,1947,NaT,NaT,1000045
2,1000104,6,1953,NaT,NaT,1000104
3,1000171,6,1945,2020-03-05 12:19:11,NaT,1000171
4,1000196,6,1944,NaT,NaT,1000196


In [52]:
measure_date3 = measure_date2.iloc[:,[0,5,1,2,3,4]]
measure_date3.head()

,FID,IID,Birth_Month,Birth_Year,Measurement_date_i0,Measurement_date_i1
0,1000028,1000028,2,1955,2019-09-25 21:18:41,2021-06-23 07:21:04
1,1000045,1000045,1,1947,NaT,NaT
2,1000104,1000104,6,1953,NaT,NaT
3,1000171,1000171,6,1945,2020-03-05 12:19:11,NaT
4,1000196,1000196,6,1944,NaT,NaT


In [53]:
measure_date3.shape

(502150, 6)

In [54]:
measure_date4 = measure_date3.dropna(subset=measure_date3.columns[3:],how='all')
measure_date4.head()

,FID,IID,Birth_Month,Birth_Year,Measurement_date_i0,Measurement_date_i1
0,1000028,1000028,2,1955,2019-09-25 21:18:41,2021-06-23 07:21:04
1,1000045,1000045,1,1947,NaT,NaT
2,1000104,1000104,6,1953,NaT,NaT
3,1000171,1000171,6,1945,2020-03-05 12:19:11,NaT
4,1000196,1000196,6,1944,NaT,NaT


In [55]:
measure_date4.shape

(502150, 6)

In [57]:
measure_date4.to_csv('month_of_birth_date_of_measurement_metabolomics.csv',sep='\t',index=False,na_rep='NA')

In [58]:
!ls *csv

app27412_20240827225957.dataset.codings.csv
app27412_20240827225957.dataset.data_dictionary.csv
app27412_20240827225957.dataset.entity_dictionary.csv
metabolomics_pheno_2nd_instance_for_pravesh.csv
month_of_birth_date_of_measurement_metabolomics.csv


In [64]:
!dx upload metabolomics_pheno_2nd_instance_for_pravesh.csv --path /FEMA_metabolomics/

[===========================================================>] Uploaded 32,987,361 of 32,987,361 bytes (100%) metabolomics_pheno_2nd_instance_for_pravesh.csv
ID                                file-Gz519J0JXXjj007pqxJX2kKz
Class                             file
Project                           project-Gq71K30JXXjz5JzVpYjXY1X6
Folder                            /FEMA_metabolomics
Name                              metabolomics_pheno_2nd_instance_for_pravesh.csv
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Thu Mar  6 21:33:40 2025
Created by                        zillur
 via the job                      job-Gz501jQJXXjQ8b0K6ZjjBq07
Last modified                     Thu Mar  6 21:33:42 2025
Media type                        
archivalState                     "live"
cloudAccoun

In [63]:
!dx upload month_of_birth_date_of_measurement_metabolomics.csv --path /FEMA_metabolomics/

[===========================================================>] Uploaded 19,627,931 of 19,627,931 bytes (100%) month_of_birth_date_of_measurement_metabolomics.csv
ID                                file-Gz5195jJXXjzy9fq7PP13JG7
Class                             file
Project                           project-Gq71K30JXXjz5JzVpYjXY1X6
Folder                            /FEMA_metabolomics
Name                              month_of_birth_date_of_measurement_metabolomics.csv
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Thu Mar  6 21:33:11 2025
Created by                        zillur
 via the job                      job-Gz501jQJXXjQ8b0K6ZjjBq07
Last modified                     Thu Mar  6 21:33:12 2025
Media type                        
archivalState                     "live"
clo